# Make MTC base network

In this notebook we will run through:

1. Read in wrangler standard network  
2. calculate NUMLANES, FT  
3. Transforming the standard network format to the MTC expected format    
4. Exporting the network to a shapefile and csvs  

In [1]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.DEBUG)

# Remote I/O

In [4]:
input_dir = "Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks/data/interim/step8_standard_format"
output_dir = input_dir

In [5]:
parameters = Parameters()

2020-10-09 00:17:02, INFO: Lasso base directory set as: z:\Data\Users\Sijia\MTC\github\Lasso


# Read Roadway and Transit Networks

In [6]:
link_file = os.path.join(input_dir, 'link.json')
node_file = os.path.join(input_dir, 'node.geojson')
shape_file = os.path.join(input_dir, 'shape.geojson')

roadway_net = RoadwayNetwork.read(
    link_file = link_file, 
    node_file = node_file, 
    shape_file = shape_file
)

2020-10-09 00:17:05, INFO: Reading from following files:
-Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks/data/interim/step8_standard_format\link.json
-Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks/data/interim/step8_standard_format\node.geojson
-Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks/data/interim/step8_standard_format\shape.geojson.
2020-10-09 00:21:11, INFO: Read 1634769 links from Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks/data/interim/step8_standard_format\link.json
2020-10-09 00:21:11, INFO: Read 644475 nodes from Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks/data/interim/step8_standard_format\node.geojson
2020-10-09 00:21:11, INFO: Read 869970 shapes from Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks/data/interim/step8_standard_format\shape.geojson


In [7]:
roadway_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry'],
      dtype='object')

# Make travel model network

In [8]:
m_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = roadway_net, 
    parameters = parameters)

2020-10-09 00:38:23, INFO: Lasso base directory set as: z:\Data\Users\Sijia\MTC\github\Lasso


In [11]:
m_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry'],
      dtype='object')

# Calculate number of lanes

In [12]:
m_net = mtc.determine_number_of_lanes(
    roadway_network_object = m_net,
    parameters = parameters,
    network_variable = 'numlanes'
)

m_net.links_df.numlanes.value_counts()

2020-10-09 00:44:35, INFO: Determining number of lanes
2020-10-09 00:44:35, INFO: Lasso base directory set as: z:\Data\Users\Sijia\MTC\github\Lasso
2020-10-09 00:49:29, INFO: Finished determining number of lanes using variable: numlanes


1    1486397
2     104207
3      22455
4      14124
5       6504
6        929
7        134
8         18
9          1
Name: numlanes, dtype: int64

In [13]:
pd.crosstab(m_net.links_df.roadway, m_net.links_df.numlanes)

numlanes,1,2,3,4,5,6,7,8,9
roadway,,,,,,,,,
cycleway,99067,36,4,0,0,0,0,0,0
footway,247365,87,27,8,2,0,0,0,0
motorway,115,576,640,1026,314,96,6,1,0
motorway_link,3561,1184,187,44,7,0,0,0,0
primary,2555,11633,7466,3151,1367,235,71,14,0
primary_link,1356,110,35,3,2,0,0,0,0
residential,503743,33033,1424,369,90,7,0,0,0
secondary,25600,25120,7300,6494,3534,435,44,2,0
secondary_link,1357,188,18,2,1,0,0,0,0


# Calculate facility type

In [14]:
m_net = mtc.calculate_facility_type(
    roadway_network_object = m_net,
    parameters = parameters,
    network_variable = 'ft'
)

m_net.links_df.ft.value_counts()

2020-10-09 00:52:54, INFO: Calculating Facility Type
2020-10-09 00:52:54, INFO: Lasso base directory set as: z:\Data\Users\Sijia\MTC\github\Lasso
2020-10-09 00:55:12, INFO: Finished calculating roadway class variable: ft


99    890153
7     538666
6      96883
5      58344
4      38332
3       6365
2       3252
1       2774
Name: ft, dtype: int64

In [15]:
pd.crosstab(m_net.links_df.roadway, m_net.links_df.ft)

ft,1,2,3,4,5,6,7,99
roadway,,,,,,,,
cycleway,0,0,0,0,0,0,0,99107
footway,0,0,0,0,0,0,0,247489
motorway,2774,0,0,0,0,0,0,0
motorway_link,0,0,4983,0,0,0,0,0
primary,0,0,0,15505,8432,2555,0,0
primary_link,0,0,0,0,0,1506,0,0
residential,0,0,0,0,0,0,538666,0
secondary,0,0,0,17933,24996,25600,0,0
secondary_link,0,0,0,0,0,1566,0,0
